# Kelly, Bet sizing

In [168]:
import numpy as np
import pandas as pd

In [141]:
from scipy.optimize import minimize_scalar, newton, minimize
from scipy.integrate import quad
from scipy.stats import norm

## Discrete

In [162]:
def optimal_kelly(odds, win_probability):
    edge = (1+odds)*(win_probability)-1
    return edge/odds

In [163]:
odds = 1
p = 0.53

In [164]:
optimal_kelly(odds, p)

0.06000000000000005

## Continious

#### Single Asset

Assuming that the probability distribution of returns is Gaussian

$$f = \frac{m}{s^2}$$

In [174]:
# from "2019-03-11" to "2020-03-10"
prices_df = pd.read_csv("sp50_1year.csv", index_col=0)

In [182]:
# Micron Technology
prices_df['MU']

Date
2019-03-11    39.029999
2019-03-12    39.250000
2019-03-13    38.830002
2019-03-14    38.410000
2019-03-15    39.540001
                ...    
2020-03-04    55.290001
2020-03-05    53.720001
2020-03-06    51.470001
2020-03-09    45.970001
2020-03-10    47.860001
Name: MU, Length: 253, dtype: float64

In [187]:
daily_log_returns = np.log(prices_df['MU']/prices_df['MU'].shift(1))
daily_log_returns.fillna(0, inplace=True)

In [188]:
daily_log_returns.describe()

count    253.000000
mean       0.000806
std        0.028288
min       -0.117552
25%       -0.015485
50%        0.000361
75%        0.018850
max        0.125235
Name: MU, dtype: float64

In [190]:
# Annualized Mean_return
mean_return = np.mean(daily_log_returns)*256
# Annualized standard dev
std = np.std(daily_log_returns)*np.sqrt(256)

In [191]:
mean_return, std

(0.20636791937171559, 0.4517090902944161)

In [192]:
risk_free = 0.01
mean_excess_return = mean_return -risk_free

In [193]:
kelly_fraction = mean_excess_return/std**2

In [194]:
kelly_fraction

0.9623939353487783

#### multiple assets, Optimal Allocation

In [204]:
from numpy.linalg import inv

$$F^* = C^{-1}M$$

In [200]:
tickers = list(prices_df)[:5]
tickers

['ADM', 'AGN', 'AIV', 'ALL', 'ALLE']

In [201]:
target = prices_df[tickers]
log_returns = np.log(target/target.shift(1)).dropna()

In [202]:
log_returns

,ADM,AGN,AIV,ALL,ALLE
Date,,,,,
2019-03-12,0.001633,0.007609,0.009189,-0.000425,0.002593
2019-03-13,0.008355,0.018174,0.000596,-0.005543,0.009302
2019-03-14,0.001847,-0.004462,0.008903,0.003202,-0.006736
2019-03-15,-0.003929,0.009633,0.000197,0.010493,-0.004064
2019-03-18,-0.016344,-0.002715,-0.019688,0.001896,0.005527
...,...,...,...,...,...
2020-03-04,0.033840,0.020626,0.034917,0.047481,0.041124
2020-03-05,-0.026771,-0.015688,-0.017306,-0.054281,-0.013765
2020-03-06,-0.018166,-0.004885,-0.011974,-0.012245,-0.023511


In [205]:
risk_free = 0.01
excess_return = log_returns - risk_free/256

In [206]:
# Annualized Excess returns mean
M = 256*excess_return.mean()

# Annulized Covariance matrix
C = 256*excess_return.cov()

# Kelly fraction
F=np.dot(inv(C), M)

In [208]:
M

ADM    -0.121036
AGN     0.266889
AIV     0.007390
ALL     0.063911
ALLE    0.295344
dtype: float64

In [207]:
F

array([-5.96507684,  2.63869778, -2.38328886,  1.05329052,  8.21475191])

#### The kelly Criterion and the Sotck market 

resource: [Betting with the Kelly criterion](https://quantdare.com/kelly-criterion/)

In [149]:
def norm_dev_integral(f, m, st):
    val,er = quad(lambda s: (s/(1+f*s))*norm.pdf(s,m,st),m-3*st,m+3*st)
    return val

In [150]:
# Reference values from Eduard Thorp's article
m = .058
s = .216

x0 = newton(norm_dev_integral, 1.0, args=(m,s))
print('Optimal Kelly fraction: {:.4f}'.format(x0))


Optimal Kelly fraction: 1.1974


I changed above a little to implement the paper as same as possible. I'm not sure this is correct

> paper: The Kelly Criterion and the Stock Market by Louis M.Rotando and Edward O. Thorp

In [144]:
def get_adjusted_sigma(mu, sigma, h):
    def func(alpha, mu=mu, sigma=sigma, h=h):
        value, error = quad(lambda s: s**2*(norm.pdf(s, mu, sigma)+h), mu-3*sigma, mu+3*sigma)
        return value-mu**2-sigma**2
    
    # use std as initial value
    answer = optimize.fsolve(func, sigma)
    return answer[0]
    
def quasi_normal_dev_integral(fraction, mu, sigma):
    
    # base_line = (B-A)
    base_line= (mu+3*sigma) - (mu-3*sigma)

    # h = area/base_line
    h = (1-(norm(0, 1).cdf(3.) - norm(0, 1).cdf(-3.)))/base_line
    
    alpha = get_adjusted_sigma(mu, sigma, h)
    
    pdf_of_return = lambda x: norm.pdf(x, mu, alpha)+h
        
    value, error = quad(lambda s:(s/(1+fraction*s)) * pdf_of_return(s), mu-3*sigma, mu+3*sigma)
    return value

In [145]:
# Reference values from Eduard Thorp's article
m = .058
s = .216
x0 = newton(quasi_normal_dev_integral,1.0,args=(m,s))
print('Optimal Kelly fraction: {:.4f}'.format(x0))


Optimal Kelly fraction: 1.1906
